In [1]:
import pandas as pd
import numpy as np

import torch

from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [4]:
df = pd.read_csv('../data/esnli_train_1.csv')
df.head()

In [11]:
df_cleaned = df.rename(columns={'Sentence1': 'premise', 'Sentence2': 'hypothesis', 'Explanation_1': 'explanation'}).drop(["WorkerId", "Sentence1_Highlighted_1", "Sentence2_Highlighted_1"], axis=1)
df_cleaned

In [27]:
label_to_id = {"entailment": 0, "neutral": 1, "contradiction": 2}
id_to_label = {v: k for k, v in label_to_id.items()}

In [31]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

# Tokenize input
premise = df['premise'][0]
hypothesis = df['hypothesis'][0]
explanation = df['explanation'][0]
actual_label = df['gold_label'][0]
encoded_input = tokenizer.encode_plus(premise, hypothesis, explanation, padding=True, truncation=True, return_tensors='pt')

labels = torch.tensor(df['gold_label'].replace(label_to_id).tolist())[0]
output = model(**encoded_input, labels=labels)

predicted_class = torch.argmax(output.logits, dim=1)

print(f"Premise: {premise}\nHypothesis: {hypothesis}\nExplanation: {explanation}\n")
print(f"True class: {actual_label}")
print(f"Predicted class: {id_to_label[predicted_class.item()]}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is training his horse for a competition.
Explanation: the person is not necessarily training his horse

True class: neutral
Predicted class: entailment
